<a href="https://colab.research.google.com/github/nikiforovanina/demand-estimation/blob/main/PyBLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install pyblp

In [45]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

In [48]:
product_data = pd.read_csv('dataPyBLP.csv', sep = ',')
product_data.sample(n=5, random_state=0)

,Unnamed: 0,ID,Wahlkreis,Land,Wahlberechtigte,Gultige,Ungultige,outside_share,product,value,CDU,SPD,Linke,Grune,FDP,AfD,culture,econ,antielite
57663,57664,12956,45,3,643,395,2.0,0.383,Grune,0.055,0.0,0.0,0.0,1.0,0.0,0.0,2.15,3.50,2.0
4504,4505,1011,2,1,1135,625,7.0,0.443,FDP,0.031,0.0,0.0,0.0,0.0,1.0,0.0,3.38,3.50,0.9
23504,23505,5398,20,2,870,443,3.0,0.487,Linke,0.029,0.0,0.0,1.0,0.0,0.0,0.0,4.92,1.25,5.4
56511,56512,12688,45,3,993,576,7.0,0.413,Grune,0.018,0.0,0.0,0.0,1.0,0.0,0.0,2.15,3.50,2.0
78338,78339,18068,61,12,906,468,8.0,0.475,Linke,0.128,0.0,0.0,1.0,0.0,0.0,0.0,4.92,1.25,5.4


In [51]:
market_1_data = product_data[product_data['ID'] == 1011]

# Display the filtered data
print(market_1_data)

      Unnamed: 0    ID  Wahlkreis  Land  Wahlberechtigte  Gultige  Ungultige  \
4500        4501  1011          2     1             1135      625        7.0   
4501        4502  1011          2     1             1135      625        7.0   
4502        4503  1011          2     1             1135      625        7.0   
4503        4504  1011          2     1             1135      625        7.0   
4504        4505  1011          2     1             1135      625        7.0   
4505        4506  1011          2     1             1135      625        7.0   

      outside_share product  value  CDU  SPD  Linke  Grune  FDP  AfD  culture  \
4500          0.443     CDU  0.245  1.0  0.0    0.0    0.0  0.0  0.0     6.00   
4501          0.443     SPD  0.130  0.0  1.0    0.0    0.0  0.0  0.0     4.15   
4502          0.443   Linke  0.033  0.0  0.0    1.0    0.0  0.0  0.0     4.92   
4503          0.443   Grune  0.096  0.0  0.0    0.0    1.0  0.0  0.0     2.15   
4504          0.443     FDP  0.031

renamings to make everything consistent with PyBLP notations in example code:

In [54]:
product_data = product_data.rename(columns={
    'value': 'market_share'
})
product_data.columns

Index(['Unnamed: 0', 'ID', 'Wahlkreis', 'Land', 'Wahlberechtigte', 'Gultige',
       'Ungultige', 'outside_share', 'product', 'market_share', 'CDU', 'SPD',
       'Linke', 'Grune', 'FDP', 'AfD', 'culture', 'econ', 'antielite'],
      dtype='object')

In [68]:
product_data['logit_delta']

0       -0.794
1       -0.627
2       -1.697
3       -1.785
4       -2.373
         ...  
85330   -1.600
85331   -0.633
85332   -1.151
85333   -1.664
85334   -2.027
Name: logit_delta, Length: 85335, dtype: float64

In [ ]:
product_data['logit_delta'] = np.log(product_data['market_share'] / product_data['outside_share'])

In [75]:
product_data = product_data.dropna(subset=['logit_delta'])
product_data = product_data[product_data['logit_delta'] >= -100]


In [82]:
statsmodels_ols = smf.ols('logit_delta ~ 1 + econ + culture', product_data)
statsmodels_results = statsmodels_ols.fit(cov_type='HC0')
statsmodels_results.summary2().tables[1]

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,-2.460,0.008,-326.087,0.000e+00,-2.475,-2.446
econ,0.041,0.002,24.526,7.729e-133,0.038,0.045
culture,0.092,0.002,49.790,0.000e+00,0.089,0.096


In [90]:
product_data.columns

Index(['Unnamed: 0', 'ID', 'Wahlkreis', 'Land', 'Wahlberechtigte', 'Gultige',
       'Ungultige', 'outside_share', 'product', 'market_share', 'CDU', 'SPD',
       'Linke', 'Grune', 'FDP', 'AfD', 'culture', 'econ', 'antielite',
       'logit_delta', 'prices', 'market_ids', 'shares'],
      dtype='object')

In [104]:
product_data['ID']

0            1
1            1
2            1
3            1
4            1
         ...  
85330    20096
85331    20096
85332    20099
85333    20099
85334    20099
Name: ID, Length: 84770, dtype: int64

same with PyBLP. the package is super sensitive to columns names, so we have to rename everything in a way PyBLP expects us to.

beware: we don't have prices --> I put econ dimension into culture

In [100]:
product_data['product_ids'] = product_data['product']


In [94]:
product_data['prices'] = product_data['econ']
product_data['market_ids'] = product_data['ID']
product_data['shares'] = product_data['market_share']
product_data['demand_instruments0'] = product_data['prices']


In [ ]:
product_data = product_data.rename(columns={
    'market': 'market_ids',
    'product': 'product_ids',
    'market_share': 'shares',
    'price_per_serving': 'prices',
})

In [95]:
ols_problem = pyblp.Problem(pyblp.Formulation('1 + prices + culture'), product_data)
ols_problem

Dimensions:
  T      N     K1    MD 
-----  -----  ----  ----
14223  84770   3     3  

Formulations:
     Column Indices:         0     1        2   
--------------------------  ---  ------  -------
X1: Linear Characteristics   1   prices  culture

In [96]:
ols_results = ols_problem.solve(method='1s')
ols_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +1.39E-22     0        +3.43E+02          +3.04E+02    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:01         1     

Beta Estimates (Robust SEs in Parentheses):
     1         prices       culture  
-----------  -----------  -----------
 -2.49E+00    +4.13E-02    +9.23E-02 
(+7.54E-03)  (+1.66E-03)  (+1.84E-03)

In [102]:
fe_problem = pyblp.Problem(pyblp.Formulation('0 + prices + culture', absorb='C(market_ids)'), product_data)
fe_problem

Dimensions:
  T      N     K1    MD    ED 
-----  -----  ----  ----  ----
14223  84770   2     2     1  

Formulations:
     Column Indices:          0        1   
--------------------------  ------  -------
X1: Linear Characteristics  prices  culture

In [103]:
fe_results = fe_problem.solve(method='1s')
fe_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +9.66E-27     0        +6.55E+00          +4.57E+00    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:01         1     

Beta Estimates (Robust SEs in Parentheses):
  prices       culture  
-----------  -----------
 +4.16E-02    +9.34E-02 
(+1.65E-03)  (+1.73E-03)

In [108]:
counterfactual_market = 1

In [109]:
fe_results.compute_elasticities(market_id=counterfactual_market)

array([[ 0.20288581, -0.03041267, -0.0037258 , -0.00954816, -0.00530454,
        -0.01414543],
       [-0.04348597,  0.11534481, -0.0037258 , -0.00954816, -0.00530454,
        -0.01414543],
       [-0.04348597, -0.03041267,  0.04833044, -0.00954816, -0.00530454,
        -0.01414543],
       [-0.04348597, -0.03041267, -0.0037258 ,  0.13620932, -0.00530454,
        -0.01414543],
       [-0.04348597, -0.03041267, -0.0037258 , -0.00954816,  0.14045295,
        -0.01414543],
       [-0.04348597, -0.03041267, -0.0037258 , -0.00954816, -0.00530454,
         0.31901453]])

In [34]:
product_data = pd.read_csv('erststimmen.csv', sep = ';', decimal=',')
product_data.sample(n=5, random_state=0)

<ipython-input-34-b46ade9ae7e3>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  product_data = pd.read_csv('erststimmen.csv', sep = ';', decimal=',')


,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehorigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte,Wahler,Ungultige,Gultige,CDU,SPD,DIE LINKE,GRuNE,CSU,FDP,AfD
52661,192,16,0,67,72,72,0,13,0,748,489,7,482,0.315,0.151,0.110,0.019,0.000,0.044,0.322
14093,49,3,1,2,0,0,0,30,0,823,452,6,446,0.258,0.422,0.092,0.018,0.000,0.043,0.157
38925,144,5,9,78,28,28,0,7172,0,398,186,3,183,0.415,0.284,0.060,0.115,0.000,0.066,0.055
19089,63,12,0,67,5705,357,5,6,0,294,193,2,191,0.304,0.120,0.157,0.005,0.000,0.063,0.319
70357,242,9,5,62,0,0,0,401,0,770,352,3,349,0.000,0.298,0.112,0.089,0.307,0.040,0.112


In [ ]:
product_data.iloc[:, -6:].sum(axis=1)

In [11]:
product_data.columns

Index(['Wahlkreis', 'Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde',
       'Gemeinde', 'Kennziffer Briefwahlzugehorigkeit', 'Wahlbezirk',
       'Bezirksart', 'Wahlberechtigte', 'Wahler', 'Ungultige', 'Gultige',
       'CDU', 'SPD', 'DIE LINKE', 'GRuNE', 'CSU', 'FDP', 'AfD', 'turnout'],
      dtype='object')

In [14]:
import matplotlib.pyplot as plt


In [ ]:
product_data[product_data['turnout'] < 0.4]

In [ ]:
plt.hist(product_data['turnout'], bins=10)  # Adjust the number of bins as needed
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Distribution of Column1')
plt.grid(True)
plt.show()

In [27]:
product_data

,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehorigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte,Wahler,Ungultige,Gultige,CDU,SPD,DIE LINKE,GRuNE,CSU,FDP,AfD,turnout,outside_share,market
0,1,1,0,1,0,0,0,1,0,1670,1018,13,1005,0.290,0.342,0.117,0.107,0.0,0.060,0.070,0.610,0.390,0
1,1,1,0,1,0,0,0,2,0,1671,808,16,792,0.263,0.375,0.120,0.061,0.0,0.062,0.098,0.484,0.516,1
2,1,1,0,1,0,0,0,3,0,1603,724,10,714,0.239,0.388,0.120,0.087,0.0,0.055,0.102,0.452,0.548,2
3,1,1,0,1,0,0,0,4,0,1497,687,10,677,0.196,0.369,0.173,0.115,0.0,0.043,0.080,0.459,0.541,3
4,1,1,0,1,0,0,0,5,0,1251,490,5,485,0.138,0.351,0.237,0.109,0.0,0.049,0.087,0.392,0.608,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49326,180,6,4,35,17,17,0,6,0,900,477,10,467,0.298,0.353,0.060,0.058,0.0,0.049,0.141,0.530,0.470,49326
49327,180,6,4,35,17,17,0,7,0,725,434,12,422,0.332,0.284,0.090,0.040,0.0,0.073,0.128,0.599,0.401,49327
49328,180,6,4,35,17,17,0,8,0,902,574,7,567,0.335,0.349,0.056,0.062,0.0,0.048,0.116,0.636,0.364,49328
49329,180,6,4,35,17,17,0,9,0,905,578,6,572,0.455,0.240,0.031,0.051,0.0,0.070,0.117,0.639,0.361,49329


In [7]:
product_data = product_data[product_data['Wahler'] > 50]
product_data = product_data[product_data['Wahlberechtigte'] > 50]

In [8]:
product_data['turnout'] = product_data['Wahler'] / product_data['Wahlberechtigte']
product_data['turnout']

0        0.610
1        0.484
2        0.452
3        0.459
4        0.392
         ...  
49326    0.530
49327    0.599
49328    0.636
49329    0.639
49330    0.620
Name: turnout, Length: 40963, dtype: float64

In [ ]:
product_data['outside_share'] = 1 - product_data['turnout']
product_data['outside_share']

In [33]:
product_data.iloc[:, -9:-3].sum(axis=1)

0        0.697
1        0.716
2        0.752
3        0.780
4        0.833
         ...  
49326    0.662
49327    0.616
49328    0.631
49329    0.509
49330    0.000
Length: 40963, dtype: float64

In [28]:
product_data.groupby('market')['value'].transform('sum')

KeyError: 'Column not found: value'

In [18]:
product_data['market'] =  product_data.index

In [31]:
product_data.columns

Index(['Wahlkreis', 'Land', 'Regierungsbezirk', 'Kreis', 'Verbandsgemeinde',
       'Gemeinde', 'Kennziffer Briefwahlzugehorigkeit', 'Wahlbezirk',
       'Bezirksart', 'Wahlberechtigte', 'Wahler', 'Ungultige', 'Gultige',
       'CDU', 'SPD', 'DIE LINKE', 'GRuNE', 'CSU', 'FDP', 'AfD', 'turnout',
       'outside_share', 'market'],
      dtype='object')

In [20]:
melted_data = pd.melt(product_data, id_vars=['market', 'outside_share'],
                      value_vars=['CDU', 'SPD', 'DIE LINKE', 'GRuNE', 'CSU', 'FDP', 'AfD'],
                      var_name='product', value_name='value')


In [21]:
melted_data

,market,outside_share,product,value
0,0,0.390,CDU,0.290
1,1,0.516,CDU,0.263
2,2,0.548,CDU,0.239
3,3,0.541,CDU,0.196
4,4,0.608,CDU,0.138
...,...,...,...,...
286736,49326,0.470,AfD,0.141
286737,49327,0.401,AfD,0.128
286738,49328,0.364,AfD,0.116
286739,49329,0.361,AfD,0.117


In [22]:
melted_data.groupby('market')['value'].transform('sum')

0         0.986
1         0.979
2         0.992
3         0.976
4         0.971
          ...  
286736    0.959
286737    0.948
286738    0.966
286739    0.963
286740    0.318
Name: value, Length: 286741, dtype: float64

In [ ]:
market_1_data.groupby('market')['value'].transform('sum')

In [25]:
market_1_data = melted_data[melted_data['market'] == 3]

# Display the filtered data
print(market_1_data)

        market  outside_share    product  value
3            3          0.541        CDU  0.196
40966        3          0.541        SPD  0.369
81929        3          0.541  DIE LINKE  0.173
122892       3          0.541      GRuNE  0.115
163855       3          0.541        CSU  0.000
204818       3          0.541        FDP  0.043
245781       3          0.541        AfD  0.080
